In [124]:
import pandas as pd

In [125]:
from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()

In [172]:
symbol = 'spx'

In [173]:
data = pd.read_csv('data/pred/pred_stw_brt_si1_' + symbol + '.csv', index_col=0, parse_dates=True)[1:]
prices = pd.read_csv('data/prices_open/prices_open_' + symbol + '.csv', index_col=0, parse_dates=True)[1:]

In [174]:
# data = data - data.min()
# data = data / data.max()
# data = data * 2 - 1

In [175]:
(prices.price_open[-1] - prices.price_open[0]) / prices.price_open[0]

0.1071988370855189

In [176]:
th = 0.1
dec_history = []
capital = 100000
assets = 0
i = 0
for s, p in zip(data.pred[:-1], prices.price_open):
    if i == 0:
        i += 1
        continue
    curr_min = data[:i].min().values
    curr_max = data[:i].max().values
    s = (s - curr_min) / curr_max
    s = s * 2 - 1
    decision = 0
    if s > th:
        decision = 1
    if s < th:
        decision = -1
    
    if decision == 1 and capital > 0:
        assets = capital / p
        capital = 0
    elif decision == -1 and assets > 0:
        capital = assets * p
        assets = 0
    dec_history.append([s, decision, capital, p, assets, (max(capital, assets * p) - 100000) / 100000, (p - prices.price_open[0]) / prices.price_open[0]])
    i += 1
final = max(capital, assets * prices.price_open[-1]) 
print(final, (final - 100000) / 100000)

102432.04681896683 0.024320468189668317


In [137]:
pd.DataFrame(dec_history, columns=['sent', 'decision', 'capital', 'price', 'assets', 'o', 'm']).replace({'decision' : {1 : 'buy', -1 : 'sell'}}).to_csv('tmp_dec_history.csv')